# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
import tensorflow
print(tensorflow.__version__)

2.13.0


In [2]:
%%capture
%pip install tensorflow_io sagemaker -U

In [3]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [4]:
role = sagemaker.get_execution_role()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::180228063476:role/service-role/AmazonSageMaker-ExecutionRole-20231023T213640


In [5]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://object-detection-project-20231024/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [6]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [7]:
# # build and push the docker image. This code can be commented out after being run once.
# # This will take around 10 mins.
# image_name = 'tf2-object-detection'
# !sh ./docker/build_and_push.sh $image_name

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [8]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

180228063476.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20231024185420


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints at the [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* [EfficientDet D1 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz)
* [SSD MobileNet V2 FPNLite 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz)
* [SSD ResNet50 V1 FPN 640x640 (RetinaNet50)](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz)
* [Faster R-CNN ResNet50 V1 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz)
* [Faster R-CNN ResNet152 V1 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz)

In the code below, the [EfficientDet D1](http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz) model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [9]:
# %%bash
# rm -rf /tmp/checkpoint source_dir/checkpoint
# mkdir /tmp/checkpoint source_dir/checkpoint
# wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
# tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

In the code below, the [SSD MobileNet V2 FPNLite](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz) model is downloaded and extracted.

In [10]:
# %%bash
# rm -rf /tmp/checkpoint source_dir/checkpoint
# mkdir /tmp/checkpoint source_dir/checkpoint
# wget -O /tmp/ssd_mobilenet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
# tar -zxvf /tmp/ssd_mobilenet.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint

In the code below, the [SSD ResNet50 V1 FPN (RetinaNet50)](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz) model is downloaded and extracted.

In [11]:
# %%bash
# rm -rf /tmp/checkpoint source_dir/checkpoint
# mkdir /tmp/checkpoint source_dir/checkpoint
# wget -O /tmp/ssd_resnet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
# tar -zxvf /tmp/ssd_resnet50.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint

In the code below, the [Faster R-CNN ResNet50 V1](http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz) model is downloaded and extracted.

In [12]:
%%bash
rm -rf /tmp/checkpoint source_dir/checkpoint
mkdir /tmp/checkpoint source_dir/checkpoint
wget -O /tmp/faster_rcnn_resnet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/faster_rcnn_resnet50.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint

--2023-10-24 19:06:43--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.63.207, 142.251.111.207, 142.251.16.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.63.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘/tmp/faster_rcnn_resnet50.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 15.0M 14s
    50K .......... .......... .......... .......... ..........  0% 29.1M 10s
   100K .......... .......... .......... .......... ..........  0% 29.0M 9s
   150K .......... .......... .......... .......... ..........  0% 20.7M 9s
   200K .......... .......... .......... .......... ..........  0% 29.6M 9s
   250K .......... .......... .......... .......... ..........  0% 27.6M 9s
   300K .......... .......... .......... ....

In the code below, the [Faster R-CNN ResNet152 V1](http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz) model is downloaded and extracted.

In [13]:
# %%bash
# rm -rf /tmp/checkpoint source_dir/checkpoint
# mkdir /tmp/checkpoint source_dir/checkpoint
# wget -O /tmp/faster_rcnn_resnet152.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
# tar -zxvf /tmp/faster_rcnn_resnet152.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint

## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [14]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "faster-rcnn-resnet50.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Using provided s3_resource


INFO:sagemaker:Creating training-job with name: tf2-object-detection-2023-10-24-19-06-47-994


2023-10-24 19:06:54 Starting - Starting the training job...
2023-10-24 19:07:24 Starting - Preparing the instances for training......
2023-10-24 19:08:20 Downloading - Downloading input data...
2023-10-24 19:08:50 Training - Downloading the training image...............
2023-10-24 19:11:26 Training - Training image download completed. Training in progress....2023-10-24 19:11:45,743 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-10-24 19:11:45,770 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-10-24 19:11:45,798 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-10-24 19:11:45,810 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "curre

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

The training was successful for the following three models. On a mAP basis, the EfficientDet D1 model performed the best.
* [EfficientDet D1 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz)
* [SSD MobileNet V2 FPNLite 640x640](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz)
* [SSD ResNet50 V1 FPN 640x640 (RetinaNet50)](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz)

### EfficientDet D1 640x640
I made the following changes to the pipeline.config included in the model's pretrianed weight.
```diff
--- efficientdet_d1_coco17_tpu-32/pipeline.config	2020-07-11 09:12:41
+++ efficientdet.config	2023-06-26 22:37:44
@@ -1,6 +1,6 @@
 model {
   ssd {
-    num_classes: 90
+    num_classes: 3
     image_resizer {
       keep_aspect_ratio_resizer {
         min_dimension: 640
@@ -131,7 +131,7 @@
   }
 }
 train_config {
-  batch_size: 128
+  batch_size: 8
   data_augmentation_options {
     random_horizontal_flip {
     }
@@ -158,20 +158,20 @@
     }
     use_moving_average: false
   }
-  fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED"
+  fine_tune_checkpoint: "checkpoint/ckpt-0"
   num_steps: 300000
   startup_delay_steps: 0.0
   replicas_to_aggregate: 8
   max_number_of_boxes: 100
   unpad_groundtruth_tensors: false
-  fine_tune_checkpoint_type: "classification"
+  fine_tune_checkpoint_type: "detection"
   use_bfloat16: true
   fine_tune_checkpoint_version: V2
 }
 train_input_reader: {
-  label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
+  label_map_path: "/opt/ml/input/data/train/label_map.pbtxt"
   tf_record_input_reader {
-    input_path: "PATH_TO_BE_CONFIGURED/train2017-?????-of-00256.tfrecord"
+    input_path: "/opt/ml/input/data/train/*.tfrecord"
   }
 }

@@ -182,10 +182,11 @@
 }

 eval_input_reader: {
-  label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
+  label_map_path: "/opt/ml/input/data/val/label_map.pbtxt"
   shuffle: false
   num_epochs: 1
   tf_record_input_reader {
-    input_path: "PATH_TO_BE_CONFIGURED/val2017-?????-of-00032.tfrecord"
+    input_path: "/opt/ml/input/data/val/*.tfrecord"
   }
 }
+
```

The learning rate increases from 0.00416 at step 100 to 0.0642 at step 2,000.

![efficientdet-lr](images/efficientdet-lr.png)

The mAP for the evaluation set is 0.08881.

![efficientdet-map](images/efficientdet-map.png)

### SSD MobileNet V2 FPNLite 640x640
I made the following changes to the pipeline.config included in the model's pretrianed weight.
```diff
--- ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config	2020-07-11 09:16:23
+++ ssd_mobilenet_v2_fpnlite.config	2023-06-26 23:03:30
@@ -1,6 +1,6 @@
 model {
   ssd {
-    num_classes: 90
+    num_classes: 8
     image_resizer {
       fixed_shape_resizer {
         height: 640
@@ -132,7 +132,7 @@
   }
 }
 train_config {
-  batch_size: 128
+  batch_size: 8
   data_augmentation_options {
     random_horizontal_flip {
     }
@@ -162,19 +162,19 @@
     }
     use_moving_average: false
   }
-  fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED"
+  fine_tune_checkpoint: "checkpoint/ckpt-0"
   num_steps: 50000
   startup_delay_steps: 0.0
   replicas_to_aggregate: 8
   max_number_of_boxes: 100
   unpad_groundtruth_tensors: false
-  fine_tune_checkpoint_type: "classification"
+  fine_tune_checkpoint_type: "detection"
   fine_tune_checkpoint_version: V2
 }
 train_input_reader {
-  label_map_path: "PATH_TO_BE_CONFIGURED"
+  label_map_path: "/opt/ml/input/data/train/label_map.pbtxt"
   tf_record_input_reader {
-    input_path: "PATH_TO_BE_CONFIGURED"
+    input_path: "/opt/ml/input/data/train/*.tfrecord"
   }
 }
 eval_config {
@@ -182,10 +182,11 @@
   use_moving_averages: false
 }
 eval_input_reader {
-  label_map_path: "PATH_TO_BE_CONFIGURED"
+  label_map_path: "/opt/ml/input/data/val/label_map.pbtxt"
   shuffle: false
   num_epochs: 1
   tf_record_input_reader {
-    input_path: "PATH_TO_BE_CONFIGURED"
+    input_path: "/opt/ml/input/data/val/*.tfrecord"
   }
 }
+
```

The learning rate increases from 0.032 at step 100 to 0.07989 at step 2,000.

![ssd-mobilenet-v2-fpnlite-lr](images/ssd-mobilenet-v2-fpnlite-lr.png)

The mAP for the evaluation set is 0.06231.

![ssd-mobilenet-v2-fpnlite-map](images/ssd-mobilenet-v2-fpnlite-map.png)

### SSD ResNet50 V1 FPN 640x640 (RetinaNet50)
I made the following changes to the pipeline.config included in the model's pretrianed weight.
```diff
--- ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config	2020-07-11 09:16:36
+++ ssd-resnet50-v1-fpn.config	2023-06-26 23:13:19
@@ -1,6 +1,6 @@
 model {
   ssd {
-    num_classes: 90
+    num_classes: 3
     image_resizer {
       fixed_shape_resizer {
         height: 640
@@ -128,7 +128,7 @@
   }
 }
 train_config {
-  batch_size: 64
+  batch_size: 8
   data_augmentation_options {
     random_horizontal_flip {
     }
@@ -158,20 +158,20 @@
     }
     use_moving_average: false
   }
-  fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED"
+  fine_tune_checkpoint: "checkpoint/ckpt-0"
   num_steps: 25000
   startup_delay_steps: 0.0
   replicas_to_aggregate: 8
   max_number_of_boxes: 100
   unpad_groundtruth_tensors: false
-  fine_tune_checkpoint_type: "classification"
+  fine_tune_checkpoint_type: "detection"
   use_bfloat16: true
   fine_tune_checkpoint_version: V2
 }
-train_input_reader {
-  label_map_path: "PATH_TO_BE_CONFIGURED"
+train_input_reader: {
+  label_map_path: "/opt/ml/input/data/train/label_map.pbtxt"
   tf_record_input_reader {
-    input_path: "PATH_TO_BE_CONFIGURED"
+    input_path: "/opt/ml/input/data/train/*.tfrecord"
   }
 }
 eval_config {
@@ -179,10 +179,11 @@
   use_moving_averages: false
 }
 eval_input_reader {
-  label_map_path: "PATH_TO_BE_CONFIGURED"
+  label_map_path: "/opt/ml/input/data/val/label_map.pbtxt"
   shuffle: false
   num_epochs: 1
   tf_record_input_reader {
-    input_path: "PATH_TO_BE_CONFIGURED"
+    input_path: "/opt/ml/input/data/val/*.tfrecord"
   }
 }
+
```

The learning rate increases from 0.01467 at step 100 to 0.4 at step 2,000.

![ssd-resnet50-v1-fpn-lr](images/ssd-resnet50-v1-fpn-lr.png)

The mAP for the evaluation set is 0.04913.

![ssd-resnet50-v1-fpn-map](images/ssd-resnet50-v1-fpn-map.png)